# <p style="text-align: center;, font-style: strong;">Partie 2 : MNIST with Multiple Layer Perceptron (MLP)</p>

### <p style="text-align: center;">(Almond 0.8.1, Scala 2.12.8)</p>


## Dependencies

In [1]:
interp.load.ivy(coursierapi.Dependency.of("org.platanios", "tensorflow_2.12", "0.4.1").withClassifier("darwin-cpu-x86_64"))
interp.load.ivy("org.platanios" %% "tensorflow-data" % "0.4.1")

In [2]:
import java.nio.file.Paths

import org.platanios.tensorflow.api._

import org.platanios.tensorflow.api.tf
import org.platanios.tensorflow.api.tensors.Tensor
import org.platanios.tensorflow.api.core.Shape
import org.platanios.tensorflow.api.core.Indexer._
import org.platanios.tensorflow.api.core.client.Session
import org.platanios.tensorflow.data.image.MNISTLoader

import org.platanios.tensorflow.api.learn.layers.{ Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean }
import org.platanios.tensorflow.api.learn.{ Model, StopCriteria }
import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator


import java.nio.file.Paths


import org.platanios.tensorflow.api._


import org.platanios.tensorflow.api.tf

import org.platanios.tensorflow.api.tensors.Tensor

import org.platanios.tensorflow.api.core.Shape

import org.platanios.tensorflow.api.core.Indexer._

import org.platanios.tensorflow.api.core.client.Session

import org.platanios.tensorflow.data.image.MNISTLoader


import org.platanios.tensorflow.api.learn.layers.{ Flatten, Input, Linear, ReLU, SparseSoftmaxCrossEntropy, Mean }

import org.platanios.tensorflow.api.learn.{ Model, StopCriteria }

import org.platanios.tensorflow.api.learn.estimators.InMemoryEstimator


## Display MNIST Dataset

In [3]:
{{
def displayNumberMNIST(nb: Int) {
    val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
    val images = dataset.trainImages
    val imagesToDisplay = images.slice(0 :: nb, ::, ::)
    for (index <- 0 until nb) {
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imagesToDisplay(index).reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}
displayNumberMNIST(20)
}}

2019-09-19 15:57:31.200 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-09-19 15:57:32.648 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-09-19 15:57:32.652 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-09-19 15:57:32.714 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-09-19 15:57:32.717 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


In [4]:
val dataset = MNISTLoader.load(Paths.get("../resources/dataset"))
val trainImages = tf.data.datasetFromTensorSlices(dataset.trainImages.toFloat)

val trainLabels = tf.data.datasetFromTensorSlices(dataset.trainLabels.toLong)
val trainData =
  trainImages.zip(trainLabels)
      .repeat()
      .shuffle(10000)
      .batch(256)
      .prefetch(10)

// Create the MLP model.
val input = Input(FLOAT32, Shape(-1, 28, 28))
val trainInput = Input(INT64, Shape(-1))
val layer = Flatten[Float]("Input/Flatten") >>
    Linear[Float]("Layer_0", 256) >> ReLU[Float]("Layer_0/Activation") >>
    Linear[Float]("Layer_1", 256) >> ReLU[Float]("Layer_1/Activation") >>
    Linear[Float]("OutputLayer", 10)
val loss = SparseSoftmaxCrossEntropy[Float, Long, Float]("Loss") >>
    Mean("Loss/Mean")
val optimizer = tf.train.Adam()
val model = Model.simpleSupervised(input, trainInput, layer, loss, optimizer)

// Create an estimator and train the model.
val estimator = InMemoryEstimator(model)
estimator.train(() => trainData, StopCriteria(maxSteps = Some(10000)))


2019-09-19 15:57:36.725 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/train-images-idx3-ubyte.gz'.
2019-09-19 15:57:37.025 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/train-labels-idx1-ubyte.gz'.
2019-09-19 15:57:37.027 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting images from file '../resources/dataset/t10k-images-idx3-ubyte.gz'.
2019-09-19 15:57:37.093 [scala-interpreter-1] INFO  MNIST Data Loader - Extracting labels from file '../resources/dataset/t10k-labels-idx1-ubyte.gz'.
2019-09-19 15:57:37.097 [scala-interpreter-1] INFO  MNIST Data Loader - Finished loading the MNIST dataset.


dataset: org.platanios.tensorflow.data.image.MNISTDataset = MNISTDataset(
  MNIST,
  Tensor[UByte, [60000, 28, 28]],
  Tensor[UByte, [60000]],
  Tensor[UByte, [10000, 28, 28]],
  Tensor[UByte, [10000]]
)
trainImages: ops.data.Dataset[Output[Float]] = Dataset[TensorSlicesDataset]
trainLabels: ops.data.Dataset[Output[Long]] = Dataset[TensorSlicesDataset]
trainData: ops.data.Dataset[(Output[Float], Output[Long])] = Dataset[TensorSlicesDataset/Zip/Repeat/Shuffle/Batch/Prefetch]
input: Input[Output[Float]] = org.platanios.tensorflow.api.learn.layers.Input@cb01a8d
trainInput: Input[Output[Long]] = org.platanios.tensorflow.api.learn.layers.Input@8fb840b
layer: learn.layers.Compose[Output[Float], Output[Float], Output[Float]] = Compose(
  "Input/Flatten",
  Compose(
    "Input/Flatten",
    Compose(
      "Input/Flatten",
      Compose(
        "Input/Flatten",
        Compose(
          "Input/Flatten",
          Flatten("Input/Flatten"),
          Linear(
            "Layer_0",
            2

In [5]:
def accuracy(images: Tensor[UByte], labels: Tensor[UByte]): Float = {
    val predictions = estimator.infer(() => images.toFloat)
    predictions
      .argmax(1).toUByte
      .equal(labels).toFloat
      .mean().scalar
}

println(s"Train accuracy = ${accuracy(dataset.trainImages, dataset.trainLabels)}")
println(s"Test accuracy = ${accuracy(dataset.testImages, dataset.testLabels)}")

Train accuracy = 0.99691665
Test accuracy = 0.9496


defined function accuracy

## Test results

In [6]:
val images = dataset.testImages

def inferOnSelectedImage(indexes: Seq[Int], images: Tensor[UByte]) {
    indexes.foreach { index => 
        val imageToInfer = images.slice(index, ::, ::).reshape(Shape(1, 28, 28))
        val predictions = estimator.infer(() => imageToInfer.toFloat)
        println(s"Label infered: ${predictions.argmax(1).scalar}")
        val png = Session().run(fetches = tf.decodeRaw[Byte](tf.image.encodePng(imageToInfer.reshape(Shape(28, 28, 1)))))
        Image(png.entriesIterator.toArray).withFormat(Image.PNG).withWidth(100).withHeight(100).display 
    }
}

inferOnSelectedImage((1 to 10), images)


Label infered: 2


Label infered: 1


Label infered: 0


Label infered: 4


Label infered: 1


...

Label infered: 9


Label infered: 6


Label infered: 9


Label infered: 0


images: Tensor[UByte] = Tensor[UByte, [10000, 28, 28]]
defined function inferOnSelectedImage